In [3]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import StratifiedShuffleSplit
#nltk.download('punkt')

In [4]:
import os
import pandas as pd
os.chdir('D:\APDS\Project\Yelp\py workspace')

In [5]:
df_recommended = pd.read_csv('..\\dataset\\RestaurantData\\recommended.csv')
df_not_recommended = pd.read_csv('..\\dataset\\RestaurantData\\not_recommended.csv')
frames = [df_recommended, df_not_recommended]
df = pd.concat(frames, keys=['1','0'])

In [6]:
df.head()

SNo               Review_Id                 User_Id  \
1 0    0  tfR6cUgapL4qa0ayD-MXJQ  YEDnTf7d6RyXKEHDGqi9tg   
  1    1  b9FySx_x4XeBXzlVU3OC9w  waaZROsfwU7mKcaW8cINtg   
  2    2  ER-O3ogvbFnKq2TUWPxhxQ  6oRhkk76MBoG3ky3AaBMsA   
  3    3  GLj88OeSVnOH48u0lQ7JBw  OucFS92pVsGBwzELoBnWvA   
  4    4  bBYMeN4NZBhkhmrIPBVEBA  KgFOJnWXxVuz0b-xfo6jjw   

                                           Review_Text  
1 0  Toronto hidden gem alert! I am so glad that I ...  
  1  We came here in search for a new Italian resta...  
  2  I had the worst service I could ever had in to...  
  3  Great experience tucked away in a side street ...  
  4  I was looking for a new Italian restaurant aro...

In [7]:
df.isnull().values.any()

False

In [31]:
def splitData(testSize):
    rawDataArr = np.array(rawData)
    X = rawDataArr[:,[0,1]]
    y = rawDataArr[:,[2]]
    sss = StratifiedShuffleSplit(n_splits=1, test_size=testSize, random_state=0)
    sss.get_n_splits(X, y)
    
    for train_index, test_index in sss.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train = np.concatenate((X_train,y_train),axis=1) #change this if number of splits>1
        test = np.concatenate((X_test,y_test),axis=1) #change this if number of splits>1
        
    for (_, Text, Label) in train[:]:
        trainData.append((toFeatureVector(preProcess(Text)),Label))
    for (_, Text, Label) in test[:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))

In [8]:
def loadData(Text=None):
    for row in df.itertuples():
        (Id, Text, Label) = parseReview(row)
        rawData.append((Id, Text, Label))
        preprocessedData.append((Id, preProcess(Text), Label))

In [9]:
def parseReview(reviewRow):
    # Should return a triple of an integer, a string containing the review, and a string indicating the label    
    return (reviewRow[2], reviewRow[4], reviewRow[0][0])

In [10]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION

# Input: a string of one review
def preProcess(text):
    # Should return a list of tokens
    return word_tokenize(text)

In [11]:
featureDict = {} # A global dictionary of features

def toFeatureVector(tokens):
    localDict = {}
    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] = +1
   
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] = +1
    
    return localDict

In [12]:
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)

In [43]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
        classifier = trainClassifier(dataset[:i]+dataset[foldSize+i:])
        y_pred = predictLabels(dataset[i:i+foldSize],classifier)
        a = accuracy_score(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred)
        (p,r,f,_) = precision_recall_fscore_support(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred, average ='macro')
        cv_results.append((a,p,r,f))
    cv_results = (np.mean(np.array(cv_results),axis=0))
    return cv_results

In [44]:
def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

def predictLabel(reviewSample, classifier):
    return classifier.classify(toFeatureVector(preProcess(reviewSample)))

In [45]:
# MAIN

# loading reviews
rawData = []          
preprocessedData = [] 
trainData = []        
testData = []         

# the output classes
fakeLabel = '0'
realLabel = '1'

print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')
loadData()
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')

splitData(0.3)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')
print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 10))

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 4419 rawData, 0 trainData, 0 testData
Preparing training and test data...
Now 4419 rawData, 3093 trainData, 1326 testData
Training Samples: 
3093
Features: 
22994
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Mean of cross-validations (Accuracy, Precision, Recall, Fscore):  [0.72903795 0.69018381 0.69746614 0.68491029]


#### INCLUDING LEMMATIZATION,REMOVING STOP WORDS AND PUNCTUATIONS

In [46]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import string

In [47]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION

table = str.maketrans({key: None for key in string.punctuation})

def preProcess(text):
    # Should return a list of tokens
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    stop_words = set(stopwords.words('english'))
    text = text.translate(table)
    for w in text.split(" "):
        if w not in stop_words:
            filtered_tokens.append(lemmatizer.lemmatize(w.lower()))
    return filtered_tokens

In [51]:
#nltk.download('stopwords')
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Sahil
[nltk_data]     Tyagi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [52]:
# MAIN

# loading reviews
rawData = []          
preprocessedData = [] 
trainData = []        
testData = []         

# the output classes
fakeLabel = '0'
realLabel = '1'

print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')

loadData()
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')

splitData(0.3)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')
print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 10))

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 4419 rawData, 0 trainData, 0 testData
Preparing training and test data...
Now 4419 rawData, 3093 trainData, 1326 testData
Training Samples: 
3093
Features: 
31458
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Mean of cross-validations (Accuracy, Precision, Recall, Fscore):  [0.77169756 0.72678067 0.73730308 0.73089252]


#### INTRODUCING THE BIGRAMS & TRYING DIFFERENT VALUES OF C IN LINEARSVC FUNCTION

In [53]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION
# Input: a string of one review
table = str.maketrans({key: None for key in string.punctuation})
def preProcess(text):
    # Should return a list of tokens
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    lemmatized_tokens = []
    stop_words = set(stopwords.words('english'))
    text = text.translate(table)
    for w in text.split(" "):
        if w not in stop_words:
            lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens
    return filtered_tokens

In [54]:
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC(C=0.01))])
    return SklearnClassifier(pipeline).train(trainData)

In [55]:
# MAIN

# loading reviews
rawData = []          
preprocessedData = [] 
trainData = []        
testData = []         

# the output classes
fakeLabel = '0'
realLabel = '1'

print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')

loadData()
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')

splitData(0.3)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')
print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 10))

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 4419 rawData, 0 trainData, 0 testData
Preparing training and test data...
Now 4419 rawData, 3093 trainData, 1326 testData
Training Samples: 
3093
Features: 
206913
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Mean of cross-validations (Accuracy, Precision, Recall, Fscore):  [0.80788467 0.77386047 0.73597209 0.74759226]


In [56]:
#  TEST DATA
classifier = trainClassifier(trainData)
predictions = predictLabels(testData, classifier)
true_labels = list(map(lambda d: d[1], testData))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.7865761689291101
Precision:  0.7450969664189486
Recall:  0.7865761689291101
f1-score:  0.7266148684828815
